In [17]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json

from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Crash Location Heatmap
Read crash locations, clean data, create heatmap

In [3]:
# Read crashes file into DataFrame
crashes = pd.read_csv("Resources/Traffic_Crashes_-_Crashes_2017-present.csv", low_memory=False)

# Display DataFrame head
crashes.head()

,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,JC193526,NaN,3/21/2019 0:50,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,2.0,0.0,0,5,3,41.808203,-87.704125,POINT (-87.704125461348 41.808203053915)
1,JC193531,NaN,3/21/2019 0:49,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,...,0.0,0.0,1.0,0.0,0,5,3,41.791284,-87.741697,POINT (-87.741696867821 41.791283705634)
2,JC193482,NaN,3/20/2019 22:35,5,TRAFFIC SIGNAL,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,RAMP,...,0.0,0.0,1.0,0.0,22,4,3,41.884390,-87.629599,POINT (-87.629599098569 41.884389603817)
3,JC193506,NaN,3/20/2019 22:00,30,NO CONTROLS,NO CONTROLS,RAIN,DARKNESS,REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,22,4,3,41.685684,-87.603466,POINT (-87.603466225555 41.685683631223)
4,JC193468,NaN,3/20/2019 22:00,30,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,22,4,3,41.910866,-87.649665,POINT (-87.649664933008 41.910865686084)


In [4]:
# Display summary of Crash DataFrame
crashes.describe()

,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,225631.000000,1.478540e+05,225631.000000,225631.000000,225104.000000,224651.000000,224651.000000,224651.000000,224651.000000,224651.000000,224651.000000,224651.0,225631.000000,225631.000000,225631.000000,224734.000000,224734.000000
mean,28.255905,1.704561e+01,3596.120954,1261.457473,2.023540,0.172107,0.001006,0.019154,0.094288,0.057658,2.022110,0.0,13.130962,4.119908,6.438712,41.859790,-87.674064
std,6.629212,3.435941e+03,2803.602162,709.865065,0.428493,0.532311,0.034782,0.161130,0.391660,0.303875,1.178442,0.0,5.474573,1.968728,3.590959,0.329580,0.669363
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.933994
25%,30.000000,2.000000e+00,1200.000000,724.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,3.000000,41.790613,-87.721502
50%,30.000000,2.000000e+00,3131.000000,1221.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.879495,-87.673883
75%,30.000000,4.000000e+00,5500.000000,1831.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.926959,-87.634074
max,70.000000,1.191625e+06,13799.000000,2535.000000,18.000000,21.000000,3.000000,7.000000,21.000000,10.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [5]:
# Filter Crash DataFrame to include Latitude and Longitude
crash_location = crashes.loc[:,"LATITUDE":"LONGITUDE"]

# Clean resulting DataFrame by removing NaN values
crash_location["LATITUDE"].dropna(how='any', inplace=True)
crash_location["LONGITUDE"].dropna(how='any', inplace=True)

# Rename column headers
crash_location.rename(columns={"LATITUDE": "Lat", "LONGITUDE":"Lng"}, inplace=True)

# Display DataFrame head
crash_location.head()

,Lat,Lng
0,41.808203,-87.704125
1,41.791284,-87.741697
2,41.884390,-87.629599
3,41.685684,-87.603466
4,41.910866,-87.649665


In [6]:
# Display summary of cleaned Crash DataFrame
crash_location.describe()

,Lat,Lng
count,224734.000000,224734.000000
mean,41.859790,-87.674064
std,0.329580,0.669363
min,0.000000,-87.933994
25%,41.790613,-87.721502
50%,41.879495,-87.673883
75%,41.926959,-87.634074
max,42.022780,0.000000


In [7]:
# Pull latitude and longitude data for Chicago from Google Maps
target_url = (f"https://maps.googleapis.com/maps/api/geocode/json?address=Chicago&key={gkey}")
geo_data = requests.get(target_url).json()
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Chicago",
                    "short_name": "Chicago",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Cook County",
                    "short_name": "Cook County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Illinois",
                    "short_name": "IL",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
                    "types": [
 

In [8]:
# Store latitude and longitude boundaries
lat_min = geo_data['results'][0]['geometry']['bounds']['southwest']['lat']
lat_max = geo_data['results'][0]['geometry']['bounds']['northeast']['lat']
lng_min = geo_data['results'][0]['geometry']['bounds']['southwest']['lng']
lng_max = geo_data['results'][0]['geometry']['bounds']['northeast']['lng']

# Display latitude and longitude boundaries
print(f"Illinois spans latitude {lat_min} to {lat_max}.")
print(f"Illinois spans longitude {lng_min} to {lng_max}.")
print("We will need to discard any errant data that falls outside these bounds.")

Illinois spans latitude 41.6443349 to 42.023131.
Illinois spans longitude -87.9402669 to -87.52366099999999.
We will need to discard any errant data that falls outside these bounds.


In [9]:
# Iterate through cleaned Crash DataFrame, discard any errant location data that falls outside the boundaries
i = 0
for index, row in crash_location.iterrows():
    if (crash_location["Lat"][index] < lat_min) | (crash_location["Lat"][index] > lat_max) | (np.isnan(crash_location["Lat"][index])) | (crash_location["Lng"][index] < lng_min) | (crash_location["Lng"][index] > lng_max) | (np.isnan(crash_location["Lng"][index])):
        print(f"Out of bounds: lat/lng {crash_location['Lat'][index]}, {crash_location['Lng'][index]}. Dropping record...")
        crash_location.drop(index, inplace=True)
        i += 1
print(f"Cleaning complete: {i} rows were dropped due to lat/lng outside bounds.")

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

In [10]:
# Display summary of cleaned Crash DataFrame
crash_location.describe()

,Lat,Lng
count,224721.000000,224721.000000
mean,41.862211,-87.679136
std,0.085184,0.058063
min,41.644670,-87.933994
25%,41.790637,-87.721508
50%,41.879495,-87.673888
75%,41.926969,-87.634080
max,42.022780,-87.524587


In [11]:
# Plot Heatmap of crashes

fig = gmaps.figure(center=(41.835966, -87.825), zoom_level=11)

heatmap_layer = gmaps.heatmap_layer(crash_location, max_intensity=900, point_radius=.01, dissipating=False)

heatmap_layer.dissipating = False
heatmap_layer.max_intensity = 1000
heatmap_layer.point_radius = .01

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Red Light Camera Markers 
Read red light camera locations, store lat/lng to overlay on crash heatmap

In [ ]:
# Read red light camera location file into DataFrame
rl_cams = pd.read_csv("Resources/Red_Light_Camera_Locations.csv", low_memory=False)

# Display DataFrame head
rl_cams.head()

In [13]:
# Filter Crash DataFrame to include Latitude and Longitude
rl_location = rl_cams.loc[:,"LATITUDE":"LONGITUDE"]

# Clean resulting DataFrame by removing NaN values
rl_location["LATITUDE"].dropna(how='any', inplace=True)
rl_location["LONGITUDE"].dropna(how='any', inplace=True)

# Rename column headers
rl_location.rename(columns={"LATITUDE": "Lat", "LONGITUDE":"Lng"}, inplace=True)

# Display DataFrame head
rl_location.head()

,Lat,Lng
0,41.931791,-87.726979
1,41.975532,-87.728234
2,41.924237,-87.746302
3,41.937997,-87.806746
4,41.923676,-87.785441


# Speed Camera Markers 
Read speed camera locations, store lat/lng to overlay on crash heatmap

In [14]:
# Read red light camera location file into DataFrame
speed_cams = pd.read_csv("Resources/Speed_Camera_Locations.csv", low_memory=False)

# Display DataFrame head
speed_cams.head()

,ADDRESS,FIRST APPROACH,SECOND APPROACH,GO-LIVE DATE,LATITUDE,LONGITUDE,LOCATION
0,3843 W 111th (Speed Camera),EB,WB,01/13/2014,41.691202,-87.717211,"(41.69120239624487, -87.71721139909997)"
1,19 W Chicago Ave (Speed Camera),WB,NaN,04/29/2014,41.896556,-87.629026,"(41.89655610710888, -87.62902590382873)"
2,2445 W 51st St (Speed Camera),EB,NaN,02/24/2014,41.801013,-87.686071,"(41.801012880525555, -87.68607060493738)"
3,7739 S Western (Speed Camera),NB,NaN,12/18/2013,41.752629,-87.682765,"(41.752629293489285, -87.68276525603093)"
4,3832 W 79th St (Speed Camera),EB,NaN,02/10/2014,41.749715,-87.719599,"(41.749715176851936, -87.71959877920995)"


In [15]:
# Filter Crash DataFrame to include Latitude and Longitude
speed_location = speed_cams.loc[:,"LATITUDE":"LONGITUDE"]

# Clean resulting DataFrame by removing NaN values
speed_location["LATITUDE"].dropna(how='any', inplace=True)
speed_location["LONGITUDE"].dropna(how='any', inplace=True)

# Rename column headers
speed_location.rename(columns={"LATITUDE": "Lat", "LONGITUDE":"Lng"}, inplace=True)

# Display DataFrame head
speed_location.head()

,Lat,Lng
0,41.691202,-87.717211
1,41.896556,-87.629026
2,41.801013,-87.686071
3,41.752629,-87.682765
4,41.749715,-87.719599


# Create Crash Heatmap with Camera Markers 
Overlay red light camera locations and speed camera locations onto crash heatmap

In [16]:
# Store intersection name for labels
rl_intersection = rl_cams.loc[:,"INTERSECTION"]

# Create red light intersection layer
rl_intersection_layer = gmaps.symbol_layer(
    rl_location, fill_color='rgba(225, 0, 0, 0.4)',
    stroke_color='rgba(225, 0, 0, 0.4)', scale=2,
    info_box_content=[f"Location: {intersection}" for intersection in rl_intersection]
)

# Create speed camera intersection layer
speed_cams_layer = gmaps.symbol_layer(
    speed_location, fill_color='rgba(0, 0, 225, 0.3)',
    stroke_color='rgba(0, 0, 225, 0.3)', scale=2,
)

# Plot Heatmap of crashes, including red light cameras and speed cameras
fig = gmaps.figure(center=(41.835966, -87.825), zoom_level=11)

fig.add_layer(heatmap_layer)
fig.add_layer(rl_intersection_layer)
fig.add_layer(speed_cams_layer)

fig 


Figure(layout=FigureLayout(height='420px'))